In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
words[:8]


['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [4]:
len(words)

32033

In [5]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [81]:
# build the dataset

block_size = 3
X, Y = [], []
for w in words:
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)

In [82]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([228146, 3]), torch.int64, torch.Size([228146]), torch.int64)

In [8]:
C = torch.randn((27, 2))
C

tensor([[-0.3625,  0.8061],
        [-0.0141,  0.3245],
        [-0.3917, -0.1756],
        [-1.2910, -0.3684],
        [ 0.0629, -0.4027],
        [ 0.2981,  0.6742],
        [ 0.6336,  2.2001],
        [ 0.2108, -1.9364],
        [-1.3755,  0.4741],
        [-1.4929, -1.1342],
        [-0.8533,  0.4857],
        [ 2.5865, -1.3124],
        [ 0.4318,  0.6035],
        [-1.2847,  0.1226],
        [ 0.3151,  0.6949],
        [-1.5113,  2.3402],
        [ 0.1657, -0.3820],
        [-0.3957, -2.8903],
        [-2.2036, -0.1891],
        [-0.2310, -0.4249],
        [-0.0513, -1.0042],
        [ 0.0159,  0.0969],
        [-0.7654, -0.5587],
        [-0.5171, -1.1641],
        [ 1.0787, -0.1647],
        [-1.3770, -1.6796],
        [ 0.3588,  0.3118]])

In [9]:
emb = C[X]

In [10]:
W1 = torch.randn((6, 100))
b1 = torch.randn((100))

In [ ]:
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)

torch.Size([32, 100])

In [19]:
W2 = torch.randn((100, 27))
b2 = torch.randn((27))

In [22]:
logits = h @ W2 + b2

In [23]:
counts = logits.exp()

In [24]:
prob = counts / counts.sum(1, keepdim=True)

In [31]:
loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(17.5572)

In [ ]:
# ------------ now made respectable :) ---------------

In [151]:
X.shape, Y.shape 

(torch.Size([228146, 3]), torch.Size([228146]))

In [174]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27, 2), generator=g)
W1 = torch.randn((6, 100), generator=g)
b1 = torch.randn((100), generator=g)
W2 = torch.randn((100, 27), generator=g)
b2 = torch.randn((27), generator=g)
parameters = [C, W1, b1, W2, b2]

In [175]:
sum(p.nelement() for p in parameters)

3481

In [176]:
for p in parameters:
    p.requires_grad = True

In [177]:
lre = torch.linspace(-3, 0, 1000)
lrs = 10**lre

In [193]:
lri = []
lossi = []

for i in range(10000):

    # minibatch construc
    ix = torch.randint(0, X.shape[0], (32, ), generator=g)
    
    # forward pass
    emb = C[X[ix]]
    h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Y[ix])
    print(loss.item())


    for p in parameters:
        p.grad = None
    loss.backward()


    lr = 0.01
    for p in parameters:
        p.data += -lr * p.grad

    # track stats
    # lri.append(lre[i])
    # lossi.append(loss.item())

2.2954022884368896
2.500267505645752
2.2848503589630127
2.5383224487304688
2.170290470123291
2.210803508758545
2.224637746810913
2.256345510482788
2.261460542678833
2.568040609359741
2.510902166366577
2.4737343788146973
2.2729556560516357
2.4703867435455322
2.407926321029663
2.857297420501709
2.0374937057495117
2.581134080886841
2.2170732021331787
1.7327463626861572
2.6514809131622314
2.2434134483337402
2.4390673637390137
2.4635066986083984
2.5062429904937744
2.3674771785736084
2.0918660163879395
2.276566982269287
2.231523275375366
2.1112403869628906
2.2335727214813232
2.6562066078186035
2.5623602867126465
2.4586877822875977
2.5099263191223145
2.179839849472046
2.4808523654937744
2.1696383953094482
1.7690449953079224
2.540221691131592
2.6145520210266113
2.360377073287964
2.145184278488159
2.5626113414764404
2.464974880218506
2.3035666942596436
2.1897430419921875
2.0393495559692383
2.3032994270324707
2.4409339427948
2.416179895401001
2.7180497646331787
2.4751248359680176
2.4523310661315

In [194]:
emb = C[X]
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Y)
loss


tensor(2.2874, grad_fn=<NllLossBackward0>)